In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
active_directory = '/content/drive/MyDrive/Desktop/DP_Finetuning_Harnet_Submission'
os.chdir(active_directory)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## IMPORT LIBRARIES

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import json
import pickle
from warnings import filterwarnings
from pandas.errors import SettingWithCopyWarning

filterwarnings("ignore", category=SettingWithCopyWarning)
filterwarnings('ignore', category=UserWarning)

## READ SUBJECT FILES

Each of the data-files contains 54 columns per row, the columns contain the following data:

-  1 timestamp (s)
- 2 activityID (see II.2. for the mapping to the activities)
- 3 heart rate (bpm)
- 4-20 IMU hand
- 21-37 IMU chest
- 38-54 IMU ankle

The IMU sensory data contains the following columns:

- 1 temperature (°C)
- 2-4 3D-acceleration data (ms-2), scale: ±16g, resolution: 13-bit
-  5-7 3D-acceleration data (ms-2), scale: ±6g, resolution: 13-bit*
- 8-10 3D-gyroscope data (rad/s)
- 11-13 3D-magnetometer data (μT)
- 14-17 orientation (invalid in this data collection)

16g acceleration hand -> indexes [4, 5, 6]

timestamp -> [0]

activity_id -> [1]

In [ ]:
def read_subject_file(file_path:str)->pd.DataFrame:
    subject_name = file_path.split('/')[-1].rstrip('.dat')
    subject = pd.read_table(file_path, header=None, sep='\s+')
    return subject, subject_name

class DataProcessor:
  def __init__(self, subject_dataframe:pd.DataFrame, subject_name:str):
    self.subject_dataframe = subject_dataframe
    self.subject_name = subject_name
    self.acc_x_col = 'acc_x'
    self.acc_y_col = 'acc_y'
    self.acc_z_col = 'acc_z'
    self.timestamp = 'timestamp'
    self.activity_id = 'activity_id'

  def _extract_data(self)->pd.DataFrame:
    """
    Extract the relevant columns from the data
    """
    # timestamp, activity_id, 16g_acc_x, 16g_acc_y, 16g_acc_z
    raw_data = self.subject_dataframe.iloc[:, [0, 1, 4, 5, 6]]
    raw_data.columns = [self.timestamp , self.activity_id , self.acc_x_col, self.acc_y_col, self.acc_z_col]
    return raw_data

  def _handle_missing_values(self, data:pd.DataFrame)->pd.DataFrame:
    """
    Used linear interpolation for the acceleration data
    """
    data[self.acc_x_col] = data[self.acc_x_col].interpolate(method='linear', limit_direction='both')
    data[self.acc_y_col] = data[self.acc_y_col].interpolate(method='linear', limit_direction='both')
    data[self.acc_z_col] = data[self.acc_z_col].interpolate(method='linear', limit_direction='both')
    return data

  def sorted_timestamps(self, data:pd.DataFrame)->pd.DataFrame:
    """
    Sort the timestamps in ascending order
    """
    data = data.sort_values(by=self.timestamp, ascending=True).reset_index(drop=True)
    return data


  def downsample_from_100_to_30hz(self, data:pd.DataFrame)->pd.DataFrame:
    """
    Downsample the data from 100Hz to 30Hz
    """

    data_copy = data.copy()
    data_copy.set_index(self.timestamp, inplace=True)
    data_copy.index = pd.to_timedelta(data_copy.index, unit="s")
    data_copy = data_copy.resample('0.0333s').agg({self.activity_id: lambda x: x.mode()[0] if len(x.mode()) > 0 else x.iloc[0],
                                        self.acc_x_col:"mean",
                                        self.acc_y_col:"mean",
                                        self.acc_z_col:"mean"}).reset_index()
    return data_copy


def activity_mapping(value:int):
  mapping_dict = {
        0: "other",
        1: "lying",
        2: "sitting",
        3: "standing",
        4: "walking",
        5: "running",
        6: "cycling",
        7: "Nordic walking",
        9: "watching TV",
        10: "computer work",
        11: "car driving",
        12: "ascending stairs",
        13: "descending stairs",
        16: "vacuum cleaning",
        17: "ironing",
        18: "folding laundry",
        19: "house cleaning",
        20: "playing soccer",
        24: "rope jumping"
   }
  return mapping_dict.get(value, 'Unknown')


def create_sliding_windows(dataframe, window_length_sec, sampling_rate_hz, step_duration_sec):
  import scipy.stats as stats


  N_FEATURES = 3
  window_length_samples = int(window_length_sec * sampling_rate_hz)
  step_duration_samples = int(step_duration_sec * sampling_rate_hz)
  windows = []
  labels = []
  for i in range(0, len(dataframe)-window_length_samples, step_duration_samples):
    x = dataframe['acc_x'].values[i: i + window_length_samples]
    y = dataframe['acc_y'].values[i: i + window_length_samples]
    z = dataframe['acc_z'].values[i: i + window_length_samples]
    window = np.array([x, y, z])
    label = dataframe['activity_id'][i: i + window_length_samples]

    windows.append(window)
    labels.append(label)

  windows = np.asarray(windows).reshape(-1, N_FEATURES, window_length_samples, )
  labels = np.asarray(labels)

  return windows, labels



def clean_window_labels(window_X, window_y):
  """
  Remove the windows that have activity id = 0 ratio > 0.5
  """
  clean_window_X = []
  clean_window_y = []

  for i in range(len(window_y)):
    if np.sum(window_y[i] == 0) / len(window_y[i]) < 0.5:
      clean_window_X.append(window_X[i])
      clean_window_y.append(window_y[i])

  clean_window_X = np.array(clean_window_X)
  clean_window_y = np.array(clean_window_y)

  return clean_window_X, clean_window_y


def majority_voting(window_y):
  """
  Most frequent activity id in a window
  """
  from scipy import stats as st

  major_window_y = st.mode(window_y, axis=1).mode
  return major_window_y


def reshaped_windows(window_X, window_y):
  """
  Reshape the windows to fit the model (n_windows, n_features, n_timestamps)
  """
  window_X = window_X.reshape(window_X.shape[0], window_X.shape[2], window_X.shape[1])
  window_y = window_y.reshape(window_y.shape[0], )
  return window_X, window_y


def activity_filter(window_X, window_y):
  """
  Filtering the activities that everybody does. (1, 2, 3, 4, 12, 13, 16, 17)
  """
  filtered_window_X = []
  filtered_window_y = []

  valid_indices = (
            (window_y == 1) | (window_y == 2) | (window_y == 3) | (window_y == 4) | (window_y == 12) | (window_y == 13) | (window_y == 16) | (window_y == 17)
        )

  filtered_window_X = window_X[valid_indices]
  filtered_window_y = window_y[valid_indices]

  return filtered_window_X, filtered_window_y



In [ ]:
file_dir = 'Protocol'
subject_raw_files = os.listdir(file_dir)

In [ ]:
subject_arr_Xs = []
subject_arr_ys = []
subject_names = []

window_length_sec = 10
step_duration_sec = 5
sampling_rate_hz = 30
overlap_ratio = round((100*(window_length_sec-step_duration_sec)/window_length_sec), 2)


print(f'WINDOW LENGTH in SAMPLES: {int(window_length_sec * sampling_rate_hz)}')
print(f'STEP DURATION in SAMPLES: {int(step_duration_sec * sampling_rate_hz)}')
print(f'OVERLAPPING WINDOW RATIO: {overlap_ratio}%')


for f in subject_raw_files:
  file_path = os.path.join(file_dir, f)
  subject, subject_name = read_subject_file(file_path)
  if subject_name != 'subject109':
    print('-----------------------------------------------------------')
    print(f'Data preprocessing is starting for {subject_name}...')
    processor = DataProcessor(subject, subject_name)
    raw_data = processor._extract_data()

    subject_df = processor._handle_missing_values(raw_data)   # Missing axes values are filled by applying linear interpolation
    subject_df = processor.sorted_timestamps(subject_df)    # Update if the timestamps is not ascending
    downsampled_subject_df = processor.downsample_from_100_to_30hz(subject_df)    # Downsample the data from 100Hz to 30Hz

    win_X, win_y = create_sliding_windows(downsampled_subject_df, window_length_sec=window_length_sec, sampling_rate_hz=sampling_rate_hz, step_duration_sec=step_duration_sec)
    clean_win_X, clean_win_y = clean_window_labels(win_X, win_y)    # Remove the windows that have activity id = 0 ratio > 0.5
    major_win_y = majority_voting(clean_win_y)    # Majority voting for the labels in a window
    filtered_window_X, filtered_window_y = activity_filter(clean_win_X, major_win_y)   # Filtering the activities that everybody does. (1, 2, 3, 4, 12, 13, 16, 17)

    print(f'Final remaining shapes X: {filtered_window_X.shape}, y: {filtered_window_y.shape}')

    subject_arr_Xs.append(filtered_window_X)
    subject_arr_ys.append(filtered_window_y)
    subject_names.append(subject_name)


all_subject_infos = list(zip(subject_names, subject_arr_Xs, subject_arr_ys))

WINDOW LENGTH in SAMPLES: 300
STEP DURATION in SAMPLES: 150
OVERLAPPING WINDOW RATIO: 50.0%
-----------------------------------------------------------
Data preprocessing is starting for subject101...
Final remaining shapes X: (345, 3, 300), y: (345,)
-----------------------------------------------------------
Data preprocessing is starting for subject102...
Final remaining shapes X: (373, 3, 300), y: (373,)
-----------------------------------------------------------
Data preprocessing is starting for subject103...
Final remaining shapes X: (348, 3, 300), y: (348,)
-----------------------------------------------------------
Data preprocessing is starting for subject104...
Final remaining shapes X: (364, 3, 300), y: (364,)
-----------------------------------------------------------
Data preprocessing is starting for subject105...
Final remaining shapes X: (379, 3, 300), y: (379,)
-----------------------------------------------------------
Data preprocessing is starting for subject106...

In [ ]:
activity_dfs = []

for sb_n, Xs, ys in all_subject_infos:
  print(f'Subject {sb_n}: {np.unique(ys, return_counts=True)}')

  subject_activities = np.vectorize(activity_mapping)(ys)
  activity_counts = pd.Series(subject_activities).value_counts(normalize=True).reset_index()
  activity_counts.columns = ['Activity', 'Ratio']
  activity_counts['Ratio'] = activity_counts['Ratio'] * 100
  activity_counts['Subject'] = sb_n

  activity_dfs.append(activity_counts)

final_activity_df = pd.concat(activity_dfs, axis=0, ignore_index=True)

Subject subject101: (array([ 1,  2,  3,  4, 12, 13, 16, 17]), array([55, 47, 43, 45, 33, 29, 46, 47]))
Subject subject102: (array([ 1,  2,  3,  4, 12, 13, 16, 17]), array([47, 45, 51, 65, 34, 31, 42, 58]))
Subject subject103: (array([ 1,  2,  3,  4, 12, 13, 16, 17]), array([44, 57, 41, 58, 21, 30, 41, 56]))
Subject subject104: (array([ 1,  2,  3,  4, 12, 13, 16, 17]), array([47, 51, 50, 64, 33, 29, 40, 50]))
Subject subject105: (array([ 1,  2,  3,  4, 12, 13, 16, 17]), array([48, 53, 45, 64, 29, 25, 49, 66]))
Subject subject106: (array([ 1,  2,  3,  4, 12, 13, 16, 17]), array([46, 46, 49, 51, 27, 22, 42, 76]))
Subject subject107: (array([ 1,  2,  3,  4, 12, 13, 16, 17]), array([51, 25, 51, 67, 36, 22, 43, 60]))
Subject subject108: (array([ 1,  2,  3,  4, 12, 13, 16, 17]), array([48, 46, 50, 63, 23, 19, 49, 66]))


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

def get_pretrained_harnet(class_num, model_name = 'harnet10'):
    repo = 'OxWearables/ssl-wearables'
    model = torch.hub.load(repo, model_name, class_num=class_num, pretrained=True, force_reload=True)
    return model

def train(model, train_loader, optimizer, criterion, epoch, device, is_dp=False, privacy_engine=None, delta=None):
  model.train()
  total_loss = 0
  correct = 0
  total_samples = 0
  for batch_idx, (data, target) in enumerate(train_loader):
      data, target = data.to(device), target.to(device)
      optimizer.zero_grad()
      output = model(data)
      loss = criterion(output, target)
      loss.backward()
      optimizer.step()
      total_loss += loss.item() * data.size(0)
      pred = output.argmax(dim=1, keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()
      total_samples += data.size(0)
      if batch_idx % 10 == 0:
          if is_dp and privacy_engine:
              epsilon = privacy_engine.get_epsilon(delta)
              print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} "
                    f"({100. * batch_idx / len(train_loader):.0f}%)]\t"
                    f"Loss: {loss.item():.6f}\tEpsilon: {epsilon:.2f}")
          else:
              print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} "
                    f"({100. * batch_idx / len(train_loader):.0f}%)]\t"
                    f"Loss: {loss.item():.6f}")

  avg_loss = total_loss / total_samples
  accuracy = 100. * correct / total_samples
  print(f"Epoch {epoch} - Training: Average loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
  return avg_loss, accuracy


def evaluate(model, test_loader, criterion, device):
  model.eval()
  test_loss = 0
  correct = 0
  all_preds = []
  all_targets = []
  with torch.no_grad():
      for data, target in test_loader:
          data, target = data.to(device), target.to(device)
          output = model(data)
          test_loss += criterion(output, target).item() * data.size(0)
          pred = output.argmax(dim=1, keepdim=True)
          correct += pred.eq(target.view_as(pred)).sum().item()
          all_preds.extend(pred.cpu().numpy())
          all_targets.extend(target.cpu().numpy())
  test_loss /= len(test_loader.dataset)
  accuracy = 100. * correct / len(test_loader.dataset)
  f1 = f1_score(all_targets, all_preds, average='macro', zero_division=0)
  print(f"Evaluation set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} "
        f"({accuracy:.2f}%), F1-score: {f1:.4f}\n")
  return test_loss, accuracy, f1


class CustomScaler:
  """
  A wrapper for scikit-learn's StandardScaler that handles both 2D and 3D NumPy arrays.
  For 3D data, it reshapes to 2D, scales, and then reshapes back.
  """
  def __init__(self):
      self._scaler = StandardScaler()
      self._is_fitted = False
      self._original_input_dims = None
  def fit(self, data: np.ndarray):
      self._original_input_dims = data.ndim
      if self._original_input_dims == 2:
          self._scaler.fit(data)
      elif self._original_input_dims == 3:
          n_samples, n_timesteps, n_features = data.shape
          reshaped_data = data.reshape((n_samples * n_timesteps, n_features))
          self._scaler.fit(reshaped_data)
      else:
          raise ValueError("Input data must have 2 or 3 dimensions for scaling.")
      self._is_fitted = True
      return self
  def fit_transform(self, data: np.ndarray) -> np.ndarray:
      self._original_input_dims = data.ndim

      if self._original_input_dims == 2:
          scaled_data = self._scaler.fit_transform(data)
      elif self._original_input_dims == 3:
          n_samples, n_timesteps, n_features = data.shape
          # Reshape 3D data to 2D for scaling
          reshaped_data = data.reshape((n_samples * n_timesteps, n_features))
          scaled_reshaped_data = self._scaler.fit_transform(reshaped_data)
          # Reshape scaled data back to original 3D shape
          scaled_data = scaled_reshaped_data.reshape((n_samples, n_timesteps, n_features))
      else:
          raise ValueError("Input data must have 2 or 3 dimensions for scaling.")

      self._is_fitted = True
      return scaled_data

  def transform(self, data: np.ndarray) -> np.ndarray:
    if not self._is_fitted:
      raise RuntimeError("Scaler has not been fitted. Call fit_transform first.")
    if data.ndim != self._original_input_dims:
      raise ValueError(f"Input data has {data.ndim} dimensions, but scaler was fitted on "
                            f"data with {self._original_input_dims} dimensions.")
    if self._original_input_dims == 2:
        scaled_data = self._scaler.transform(data)
    elif self._original_input_dims == 3:
        n_samples, n_timesteps, n_features = data.shape
        reshaped_data = data.reshape((n_samples * n_timesteps, n_features))
        scaled_reshaped_data = self._scaler.transform(reshaped_data)
        scaled_data = scaled_reshaped_data.reshape((n_samples, n_timesteps, n_features))
    else:
        raise ValueError("Input data must have 2 or 3 dimensions.")

    return scaled_data

def get_data_loader(X_test, y_test, BATCH_SIZE, shuffle=False):
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
  y_test_tensor = torch.tensor(y_test, dtype=torch.long)
  test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
  test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=shuffle)
  return test_loader

class EarlyStopping:
  def __init__(self, patience=5, delta=0, verbose=False):
      self.patience = patience
      self.delta = delta
      self.best_score = None
      self.early_stop = False
      self.counter = 0
      self.best_model_state = None
      self.verbose = verbose # Added verbose
      self.best_epoch = 0 # To track the epoch of the best model
  def __call__(self, val_loss, model, epoch):
      score = -val_loss
      if self.best_score is None:
          self.best_score = score
          self.best_model_state = copy.deepcopy(model.state_dict())
          self.best_epoch = epoch # Store best epoch
      elif score < self.best_score + self.delta:
          self.counter += 1
          if self.verbose:
              print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
          if self.counter >= self.patience:
              self.early_stop = True
      else:
          self.best_score = score
          self.best_model_state = copy.deepcopy(model.state_dict())
          self.best_epoch = epoch # Store best epoch
          self.counter = 0

  def load_best_model(self, model):
      if self.best_model_state:
          model.load_state_dict(self.best_model_state)



def plot_distribution(arr:np.ndarray, title:str):
  pd.Series(arr).value_counts(normalize=True).plot(kind='bar')
  plt.title(title)
  plt.show()


In [ ]:
def fold_plot(fold_train_accuracies, fold_val_accuracies, val_subjects):
  import matplotlib.pyplot as plt
  import math
  # Get the list of train and val accuracies
  acc_list_zipped = list(zip(fold_train_accuracies, fold_val_accuracies, val_subjects))

  # Get the total number of folds to plot
  num_folds = len(acc_list_zipped)

  # --- Subplot Layout Calculation ---
  ncols = 2
  nrows = math.ceil(num_folds / ncols)

  # --- Create the Figure and Subplots ---
  fig, axes = plt.subplots(nrows, ncols, figsize=(14, 5 * nrows))
  fig.suptitle('Training vs. Validation Accuracy Across Folds', fontsize=16, y=1.02)
  axes = axes.flatten()


  # --- Loop and Plot on Each Subplot ---
  for i, (tr_acc, vl_acc, vl_subjects) in enumerate(acc_list_zipped):
      ax = axes[i] # Get the current axis
      ax.plot(tr_acc, label='Train Accuracy', color='royalblue')
      ax.plot(vl_acc, label='Validation Accuracy', color='darkorange')
      ax.set_title(f'Fold: {i + 1} for  Validation Subjects {vl_subjects}')
      ax.set_xlabel('Epoch')
      ax.set_ylabel('Accuracy')
      ax.legend()
      ax.grid(True, linestyle='--', alpha=0.6)

  # --- Clean Up and Display ---
  # If the number of folds is odd, the last subplot in the grid will be empty.
  # This loop hides any unused subplots.
  for i in range(num_folds, len(axes)):
      axes[i].axis('off')

  # Adjusts subplot params so that subplots are nicely fit in the figure.
  fig.tight_layout(rect=[0, 0, 1, 0.98])

  return fig


def plot_epochs(train_accs, val_accs, val_subj):
  import matplotlib.pyplot as plt
  epochs = np.arange(1, len(train_accs) + 1)
  fig, ax = plt.subplots()
  ax.plot(epochs, train_accs, label='Train Accuracy', color='royalblue')
  ax.plot(epochs, val_accs, label='Validation Accuracy', color='darkorange')
  ax.set_title(f'Training vs. Validation Accuracy for Validation Subjects {val_subj}')
  ax.set_xlabel('Epoch')
  ax.set_ylabel('Accuracy')
  ax.set_xticks(epochs)
  ax.legend()
  ax.grid(True, linestyle='--', alpha=0.6)
  return fig


def remove_module_prefix(state_dict):
    return {k.replace("_module.", ""): v for k, v in state_dict.items()}


## FINE TUNING of CLASSIFIER HEADS (DP)



In [ ]:
#%pip install opacus

In [ ]:
# HYPERPARAMETER TUNING
import datetime as dt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedGroupKFold, LeaveOneGroupOut
import torch.nn as nn
import copy
from itertools import product
from opacus import PrivacyEngine
from opacus.accountants.utils import get_noise_multiplier

torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

reports_save = True

label_encoder = LabelEncoder()

X_combined = []
y_combined = []
subject_groups_combined = []

mia_test_X = []
mia_test_y = []
test_groups_combined = []
mia_test_subjects = []

for sb_name, Xs, ys in all_subject_infos:
  if sb_name not in ['subject102','subject103']:           # Test subject for membership inference attack as a non-member
    X_combined.extend(Xs)
    y_combined.extend(ys)
    subject_groups_combined.extend([sb_name] * len(ys))
  else:
    mia_test_subjects.append(sb_name)
    mia_test_X.extend(Xs)
    mia_test_y.extend(ys)
    test_groups_combined.extend([sb_name] * len(ys))



X_combined = np.array(X_combined)
y_combined = np.array(y_combined)
subject_groups_combined = np.array(subject_groups_combined)

label_encoder.fit(y_combined)
encoded_y_combined_raw = label_encoder.transform(y_combined)

EPOCHS = 20
BATCH_SIZE_LIST = [32]
LRATE_LIST = [1e-3]


parameter_pairs = list(product(BATCH_SIZE_LIST, LRATE_LIST))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
N_CLASSES = len(label_encoder.classes_)
g_noise_multp = 1.0
C_value = 1.0
noise_scale = g_noise_multp * C_value
DELTA = 1e-5

original_model = get_pretrained_harnet(class_num=N_CLASSES, model_name='harnet10')

Downloading: "https://github.com/OxWearables/ssl-wearables/zipball/main" to /root/.cache/torch/hub/main.zip


131 Weights loaded


In [ ]:


final_best_model = None
final_best_val_loss = float('inf')
final_best_val_acc = 0
final_best_val_f1 = 0
final_scaler = None
final_best_epoch = None
final_best_train_acc = None
final_best_train_loss = None
final_best_train_subjects = None
final_best_val_subjects = None
final_epsilon_spent = None
final_BATCH_SIZE = None
final_LRATE = None

for BATCH_SIZE, LRATE in parameter_pairs:
  print('-------------------------------------------------------------------------------------------------')
  print(f'Batch size: {BATCH_SIZE}, Learning rate: {LRATE}')
  print('-------------------------------------------------------------------------------------------------')

  fold_details = {"Val Subjects": [], "Batch_Size": BATCH_SIZE, "Learning_Rate": LRATE, "Epoch_Results": []}

  # Leave One Group Out for inner fold
  inner_skf = StratifiedGroupKFold(n_splits=3, shuffle=True, random_state=42)
  for fold, (train_index, val_index) in enumerate(inner_skf.split(X_combined, encoded_y_combined_raw, subject_groups_combined), start=1):
    print(f'Fold {fold} is starting ...')
    X_train, X_val = X_combined[train_index], X_combined[val_index]
    y_train, y_val = encoded_y_combined_raw[train_index], encoded_y_combined_raw[val_index]
    val_subjects = np.unique(subject_groups_combined[val_index])
    train_subjects = np.unique(subject_groups_combined[train_index])
    print(f'Validation subjects : {val_subjects}')
    print(f'Train subjects: {train_subjects}')


    scaler = CustomScaler()
    scaled_X_train = scaler.fit_transform(X_train)
    scaled_X_val = scaler.transform(X_val)
    train_data_loader = get_data_loader(scaled_X_train, y_train, BATCH_SIZE=BATCH_SIZE, shuffle=True)
    val_data_loader = get_data_loader(scaled_X_val, y_val, BATCH_SIZE=BATCH_SIZE, shuffle=False)
    model = copy.deepcopy(original_model)
    model.to(device)
    for name, param in model.named_parameters():
      if name.startswith('classifier'):
        param.requires_grad = True
      else:
        param.requires_grad = False
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LRATE, weight_decay=0.01)
    criterion = nn.CrossEntropyLoss()
    early_stopping = EarlyStopping(patience=10, verbose=True)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)
    privacy_engine = PrivacyEngine(secure_mode=False)
    model, optimizer, train_data_loader = privacy_engine.make_private(
        module=model,
        optimizer=optimizer,
        data_loader=train_data_loader,
        noise_multiplier=g_noise_multp,
        max_grad_norm=C_value
        )

    val_losses = []
    val_accuracies = []
    val_f1s = []
    train_losses = []
    train_accuracies = []
    epochs_list = []
    epoch_results = {"epoch": [], "val_subjects":'-'.join(val_subjects), "noise_scale":[], "achieved_epsilon": [],"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": [], "val_f1": []}
    best_val_loss = float('inf')
    best_model_state = None
    best_scaler = None
    best_epoch = None
    for epoch in range(1, EPOCHS + 1):
      train_loss, train_acc = train(model, train_data_loader, optimizer, criterion, epoch, device)
      val_loss, val_acc, val_f1 = evaluate(model, val_data_loader, criterion, device)
      scheduler.step(val_loss)
      train_losses.append(train_loss)
      train_accuracies.append(train_acc)
      val_losses.append(val_loss)
      val_accuracies.append(val_acc)
      val_f1s.append(val_f1)
      epochs_list.append(epoch)

      a_epsilon = round(privacy_engine.get_epsilon(DELTA), 3)
      epoch_results["noise_scale"].append(noise_scale)
      epoch_results["achieved_epsilon"].append(a_epsilon)
      print(f'Epsilon Spent: {a_epsilon}, Noise_Scale: {noise_scale}')
      epoch_results["epoch"].append(epoch)
      epoch_results["train_loss"].append(train_loss)
      epoch_results["train_acc"].append(train_acc)
      epoch_results["val_loss"].append(val_loss)
      epoch_results["val_acc"].append(val_acc)
      epoch_results["val_f1"].append(val_f1)
      if val_loss < final_best_val_loss:
        final_best_val_loss = val_loss
        final_best_model = copy.deepcopy(model.state_dict())
        final_best_val_acc = val_acc
        final_best_val_f1 = val_f1
        final_scaler = copy.deepcopy(scaler)
        final_best_epoch = epoch
        final_best_train_acc = train_acc
        final_best_train_loss = train_loss
        final_best_train_subjects = train_subjects
        final_best_val_subjects = val_subjects
        final_epsilon_spent = a_epsilon
        final_BATCH_SIZE = BATCH_SIZE
        final_LRATE = LRATE


      early_stopping(val_loss, model, epoch)
      if early_stopping.early_stop:
        print("Early stopping")
        break

    fold_details["Val Subjects"].append(val_subjects)
    fold_details["Epoch_Results"].append(epoch_results)


today_time = dt.date.today().strftime("%Y%m%d")
vers = f'{window_length_sec}_{sampling_rate_hz}_{overlap_ratio}_{today_time}'


fold_details_df = pd.DataFrame(fold_details)

noise_scale_vers = int(noise_scale)

final_model_version = f'final_model_{noise_scale_vers}_{vers}.pth'
final_scaler_version = f'final_scaler_{noise_scale_vers}_{vers}.pkl'
final_model_df = pd.DataFrame({"Final_Model_Version":[final_model_version],
                               "Final_Scaler_Version":[final_scaler_version],
                               "Final_Best_Val_Loss":final_best_val_loss,
                               "Final_Best_Val_Acc":final_best_val_acc,
                               "Final_Best_Val_F1":final_best_val_f1,
                               "Final_Best_Epoch":final_best_epoch,
                               "Final_Best_Train_Acc":final_best_train_acc,
                               "Final_Best_Train_Loss":final_best_train_loss,
                               "Final_Best_Train_Subjects":'-'.join(final_best_train_subjects),
                               "Final_Best_Val_Subjects":'-'.join(final_best_val_subjects),
                               "Final_Epsilon_Spent":final_epsilon_spent,
                               "Final_Batch_Size":final_BATCH_SIZE,
                               "Final_Learning_Rate":final_LRATE,
                               })



if reports_save:
  directory_name = 'attack_results_DP'
  os.makedirs(directory_name, exist_ok=True)

  model_directory_name = 'attack_results_DP/final_models'
  os.makedirs(model_directory_name, exist_ok=True)


  torch.save(final_best_model, os.path.join(model_directory_name, final_model_version))


  with open(os.path.join(model_directory_name, final_scaler_version), 'wb') as f:
    pickle.dump(final_scaler, f)


  final_label_encoder_version = f'final_label_encoder_{noise_scale_vers}_{vers}.pkl'
  with open(os.path.join(model_directory_name, final_label_encoder_version), 'wb') as f:
    pickle.dump(label_encoder, f)

  fold_details_df.to_csv(f'{directory_name}/fold_details_DP_CH_NS_{noise_scale_vers}_{vers}_Split_4-2-2.csv')
  final_model_df.to_csv(f'{directory_name}/final_model_df_DP_CH_NS_{noise_scale_vers}_{vers}_Split_4-2-2.csv')


-------------------------------------------------------------------------------------------------
Batch size: 32, Learning rate: 0.001
-------------------------------------------------------------------------------------------------
Fold 1 is starting ...
Validation subjects : ['subject104' 'subject105']
Train subjects: ['subject101' 'subject106' 'subject107' 'subject108']
Train Epoch: 1 [0/1423 (0%)]	Loss: 7.469398
Train Epoch: 1 [210/1423 (22%)]	Loss: 6.490749
Train Epoch: 1 [600/1423 (44%)]	Loss: 5.468999
Train Epoch: 1 [1140/1423 (67%)]	Loss: 4.276531
Train Epoch: 1 [1240/1423 (89%)]	Loss: 3.589302
Epoch 1 - Training: Average loss: 5.5608, Accuracy: 24.38%
Evaluation set: Average loss: 4.6077, Accuracy: 243/743 (32.71%), F1-score: 0.2473

Epsilon Spent: 1.235, Noise_Scale: 1.0
Train Epoch: 2 [0/1423 (0%)]	Loss: 3.525553
Train Epoch: 2 [320/1423 (22%)]	Loss: 5.900445
Train Epoch: 2 [640/1423 (44%)]	Loss: 3.040868
Train Epoch: 2 [840/1423 (67%)]	Loss: 3.475111
Train Epoch: 2 [920/142

## EVALUATION ON TEST DATA (Subject 2 & Subject 3)

In [ ]:
import torch
import gc
import os

# Enable detailed CUDA error reporting for debugging
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Clear CUDA cache first
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()

final_model_path = f'attack_results_DP/final_models/{final_model_version}'
final_scaler_path = f'attack_results_DP/final_models/{final_scaler_version}'
final_label_encoder_path = f'attack_results_DP/final_models/{final_label_encoder_version}'

criterion = nn.CrossEntropyLoss()

try:
    original_model = get_pretrained_harnet(class_num=N_CLASSES, model_name='harnet10')
    final_test_model = copy.deepcopy(original_model)

    # CRITICAL FIX: Load to CPU first, then move to device
    print("Loading model state dict...")
    state_dict = torch.load(final_model_path, weights_only=True, map_location='cpu')
    print("Applying state dict...")
    final_test_model.load_state_dict(remove_module_prefix(state_dict))
    print("Moving model to device...")
    final_test_model.to(device)

    print("Loading scaler and label encoder...")
    final_scaler_test = pickle.load(open(final_scaler_path, 'rb'))
    final_le_test = pickle.load(open(final_label_encoder_path, 'rb'))

    print("Preparing test data...")
    X_test_mia = np.array(mia_test_X)
    y_test_mia = np.array(mia_test_y)

    encoded_y_test_mia = final_le_test.transform(y_test_mia)

    # Debug info
    print(f"Original test labels: {np.unique(y_test_mia)}")
    print(f"Encoded test labels: {np.unique(encoded_y_test_mia)}")
    print(f"Label range: {np.min(encoded_y_test_mia)} to {np.max(encoded_y_test_mia)}")
    print(f"N_CLASSES: {N_CLASSES}")

    scaled_X_test_mia = final_scaler_test.transform(X_test_mia)
    mia_test_data_loader = get_data_loader(scaled_X_test_mia, encoded_y_test_mia, BATCH_SIZE=BATCH_SIZE_LIST[0], shuffle=False)

    print("Running evaluation...")
    final_test_loss, final_test_acc, final_test_f1 = evaluate(final_test_model, mia_test_data_loader, criterion, device)

    print(f'Final Model Test Loss: {final_test_loss}, Test Acc: {final_test_acc}, Test F1: {final_test_f1}')

except RuntimeError as e:
    print(f"CUDA Error occurred: {e}")
    print("Attempting CPU fallback...")

    # Fallback to CPU
    device_cpu = torch.device('cpu')

    original_model = get_pretrained_harnet(class_num=N_CLASSES, model_name='harnet10')
    final_test_model = copy.deepcopy(original_model)

    state_dict = torch.load(final_model_path, weights_only=True, map_location='cpu')
    final_test_model.load_state_dict(remove_module_prefix(state_dict))
    final_test_model.to(device_cpu)  # Use CPU

    final_scaler_test = pickle.load(open(final_scaler_path, 'rb'))
    final_le_test = pickle.load(open(final_label_encoder_path, 'rb'))

    X_test_mia = np.array(mia_test_X)
    y_test_mia = np.array(mia_test_y)
    encoded_y_test_mia = final_le_test.transform(y_test_mia)
    scaled_X_test_mia = final_scaler_test.transform(X_test_mia)

    mia_test_data_loader = get_data_loader(scaled_X_test_mia, encoded_y_test_mia, BATCH_SIZE=BATCH_SIZE_LIST[0], shuffle=False)

    final_test_loss, final_test_acc, final_test_f1 = evaluate(final_test_model, mia_test_data_loader, criterion, device_cpu)

    print(f'Final Model Test Loss (CPU): {final_test_loss}, Test Acc: {final_test_acc}, Test F1: {final_test_f1}')

Downloading: "https://github.com/OxWearables/ssl-wearables/zipball/main" to /root/.cache/torch/hub/main.zip


131 Weights loaded
Loading model state dict...
Applying state dict...
Moving model to device...
Loading scaler and label encoder...
Preparing test data...
Original test labels: [ 1  2  3  4 12 13 16 17]
Encoded test labels: [0 1 2 3 4 5 6 7]
Label range: 0 to 7
N_CLASSES: 8
Running evaluation...
Evaluation set: Average loss: 2.2889, Accuracy: 427/721 (59.22%), F1-score: 0.5614

Final Model Test Loss: 2.2888554999715907, Test Acc: 59.22330097087379, Test F1: 0.5614112570604692


## FINE TUNING of CLASSIFIER HEADS (no DP)

In [ ]:

import datetime as dt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedGroupKFold, LeaveOneGroupOut
import torch.nn as nn
import copy
from itertools import product


torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

reports_save = True

label_encoder = LabelEncoder()

X_combined = []
y_combined = []
subject_groups_combined = []

mia_test_X = []
mia_test_y = []
test_groups_combined = []
mia_test_subjects = []

for sb_name, Xs, ys in all_subject_infos:
  if sb_name not in ['subject102','subject103']:           # Test subject for membership inference attack as a non-member
    X_combined.extend(Xs)
    y_combined.extend(ys)
    subject_groups_combined.extend([sb_name] * len(ys))
  else:
    mia_test_subjects.append(sb_name)
    mia_test_X.extend(Xs)
    mia_test_y.extend(ys)
    test_groups_combined.extend([sb_name] * len(ys))



X_combined = np.array(X_combined)
y_combined = np.array(y_combined)
subject_groups_combined = np.array(subject_groups_combined)

label_encoder.fit(y_combined)
encoded_y_combined_raw = label_encoder.transform(y_combined)

EPOCHS = 20
BATCH_SIZE_LIST = [32]
LRATE_LIST = [1e-3]


parameter_pairs = list(product(BATCH_SIZE_LIST, LRATE_LIST))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
N_CLASSES = len(label_encoder.classes_)
g_noise_multp = 1.0
C_value = 1.0
noise_scale = g_noise_multp * C_value
DELTA = 1e-5

original_model = get_pretrained_harnet(class_num=N_CLASSES, model_name='harnet10')

Downloading: "https://github.com/OxWearables/ssl-wearables/zipball/main" to /root/.cache/torch/hub/main.zip


131 Weights loaded


In [ ]:
final_best_model = None
final_best_val_loss = float('inf')
final_best_val_acc = 0
final_best_val_f1 = 0
final_scaler = None
final_best_epoch = None
final_best_train_acc = None
final_best_train_loss = None
final_best_train_subjects = None
final_best_val_subjects = None
final_epsilon_spent = None
final_BATCH_SIZE = None
final_LRATE = None

for BATCH_SIZE, LRATE in parameter_pairs:
  print('-------------------------------------------------------------------------------------------------')
  print(f'Batch size: {BATCH_SIZE}, Learning rate: {LRATE}')
  print('-------------------------------------------------------------------------------------------------')

  fold_details = {"Val Subjects": [], "Batch_Size": BATCH_SIZE, "Learning_Rate": LRATE, "Epoch_Results": []}

  # Leave One Group Out for inner fold
  inner_skf = StratifiedGroupKFold(n_splits=3, shuffle=True, random_state=42)
  for fold, (train_index, val_index) in enumerate(inner_skf.split(X_combined, encoded_y_combined_raw, subject_groups_combined), start=1):
    print(f'Fold {fold} is starting ...')
    X_train, X_val = X_combined[train_index], X_combined[val_index]
    y_train, y_val = encoded_y_combined_raw[train_index], encoded_y_combined_raw[val_index]
    val_subjects = np.unique(subject_groups_combined[val_index])
    train_subjects = np.unique(subject_groups_combined[train_index])
    print(f'Validation subjects : {val_subjects}')
    print(f'Train subjects: {train_subjects}')


    scaler = CustomScaler()
    scaled_X_train = scaler.fit_transform(X_train)
    scaled_X_val = scaler.transform(X_val)
    train_data_loader = get_data_loader(scaled_X_train, y_train, BATCH_SIZE=BATCH_SIZE, shuffle=True)
    val_data_loader = get_data_loader(scaled_X_val, y_val, BATCH_SIZE=BATCH_SIZE, shuffle=False)
    model = copy.deepcopy(original_model)
    model.to(device)
    for name, param in model.named_parameters():
      if name.startswith('classifier'):
        param.requires_grad = True
      else:
        param.requires_grad = False
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LRATE, weight_decay=0.01)
    criterion = nn.CrossEntropyLoss()
    early_stopping = EarlyStopping(patience=10, verbose=True)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

    val_losses = []
    val_accuracies = []
    val_f1s = []
    train_losses = []
    train_accuracies = []
    epochs_list = []
    epoch_results = {"epoch": [], "val_subjects":'-'.join(val_subjects), "train_loss": [], "train_acc": [], "val_loss": [], "val_acc": [], "val_f1": []}
    best_val_loss = float('inf')
    best_model_state = None
    best_scaler = None
    best_epoch = None
    for epoch in range(1, EPOCHS + 1):
      train_loss, train_acc = train(model, train_data_loader, optimizer, criterion, epoch, device)
      val_loss, val_acc, val_f1 = evaluate(model, val_data_loader, criterion, device)
      scheduler.step(val_loss)
      train_losses.append(train_loss)
      train_accuracies.append(train_acc)
      val_losses.append(val_loss)
      val_accuracies.append(val_acc)
      val_f1s.append(val_f1)
      epochs_list.append(epoch)

      epoch_results["epoch"].append(epoch)
      epoch_results["train_loss"].append(train_loss)
      epoch_results["train_acc"].append(train_acc)
      epoch_results["val_loss"].append(val_loss)
      epoch_results["val_acc"].append(val_acc)
      epoch_results["val_f1"].append(val_f1)
      if val_loss < final_best_val_loss:
        final_best_val_loss = val_loss
        final_best_model = copy.deepcopy(model.state_dict())
        final_best_val_acc = val_acc
        final_best_val_f1 = val_f1
        final_scaler = copy.deepcopy(scaler)
        final_best_epoch = epoch
        final_best_train_acc = train_acc
        final_best_train_loss = train_loss
        final_best_train_subjects = train_subjects
        final_best_val_subjects = val_subjects
        final_BATCH_SIZE = BATCH_SIZE
        final_LRATE = LRATE


      early_stopping(val_loss, model, epoch)
      if early_stopping.early_stop:
        print("Early stopping")
        break

    fold_details["Val Subjects"].append(val_subjects)
    fold_details["Epoch_Results"].append(epoch_results)


today_time = dt.date.today().strftime("%Y%m%d")
vers = f'{window_length_sec}_{sampling_rate_hz}_{overlap_ratio}_{today_time}'


fold_details_df = pd.DataFrame(fold_details)

noise_scale_vers = int(noise_scale)

final_model_version = f'final_model_{vers}.pth'
final_scaler_version = f'final_scaler_{vers}.pkl'
final_model_df = pd.DataFrame({"Final_Model_Version":[final_model_version],
                               "Final_Scaler_Version":[final_scaler_version],
                               "Final_Best_Val_Loss":final_best_val_loss,
                               "Final_Best_Val_Acc":final_best_val_acc,
                               "Final_Best_Val_F1":final_best_val_f1,
                               "Final_Best_Epoch":final_best_epoch,
                               "Final_Best_Train_Acc":final_best_train_acc,
                               "Final_Best_Train_Loss":final_best_train_loss,
                               "Final_Best_Train_Subjects":'-'.join(final_best_train_subjects),
                               "Final_Best_Val_Subjects":'-'.join(final_best_val_subjects),
                               "Final_Epsilon_Spent":final_epsilon_spent,
                               "Final_Batch_Size":final_BATCH_SIZE,
                               "Final_Learning_Rate":final_LRATE,
                               })



if reports_save:
  directory_name = 'attack_results_noDP'
  os.makedirs(directory_name, exist_ok=True)

  model_directory_name = 'attack_results_noDP/final_models'
  os.makedirs(model_directory_name, exist_ok=True)


  torch.save(final_best_model, os.path.join(model_directory_name, final_model_version))


  with open(os.path.join(model_directory_name, final_scaler_version), 'wb') as f:
    pickle.dump(final_scaler, f)


  final_label_encoder_version = f'final_label_encoder_{vers}.pkl'
  with open(os.path.join(model_directory_name, final_label_encoder_version), 'wb') as f:
    pickle.dump(label_encoder, f)

  fold_details_df.to_csv(f'{directory_name}/fold_details_noDP_CH_NS_{vers}_Split_4-2-2.csv')
  final_model_df.to_csv(f'{directory_name}/final_model_df_noDP_CH_NS_{vers}_Split_4-2-2.csv')

-------------------------------------------------------------------------------------------------
Batch size: 32, Learning rate: 0.001
-------------------------------------------------------------------------------------------------
Fold 1 is starting ...
Validation subjects : ['subject104' 'subject105']
Train subjects: ['subject101' 'subject106' 'subject107' 'subject108']
Train Epoch: 1 [0/1423 (0%)]	Loss: 6.828506
Train Epoch: 1 [320/1423 (22%)]	Loss: 3.129222
Train Epoch: 1 [640/1423 (44%)]	Loss: 2.524057
Train Epoch: 1 [960/1423 (67%)]	Loss: 1.817114
Train Epoch: 1 [1280/1423 (89%)]	Loss: 2.530724
Epoch 1 - Training: Average loss: 2.8115, Accuracy: 64.65%
Evaluation set: Average loss: 2.6047, Accuracy: 509/743 (68.51%), F1-score: 0.6592

Train Epoch: 2 [0/1423 (0%)]	Loss: 1.020925
Train Epoch: 2 [320/1423 (22%)]	Loss: 2.046731
Train Epoch: 2 [640/1423 (44%)]	Loss: 1.770709
Train Epoch: 2 [960/1423 (67%)]	Loss: 0.839329
Train Epoch: 2 [1280/1423 (89%)]	Loss: 1.317361
Epoch 2 - Train

## EVALUATION On TEST DATA (SUBJECT 5 & SUBJECT 1) No DP

In [ ]:
import torch
import gc
import os

# Enable detailed CUDA error reporting for debugging
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Clear CUDA cache first
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()

final_model_path = f'attack_results_noDP/final_models/{final_model_version}'
final_scaler_path = f'attack_results_noDP/final_models/{final_scaler_version}'
final_label_encoder_path = f'attack_results_noDP/final_models/{final_label_encoder_version}'

criterion = nn.CrossEntropyLoss()

try:
    original_model = get_pretrained_harnet(class_num=N_CLASSES, model_name='harnet10')
    final_test_model = copy.deepcopy(original_model)

    # CRITICAL FIX: Load to CPU first, then move to device
    print("Loading model state dict...")
    state_dict = torch.load(final_model_path, weights_only=True, map_location='cpu')
    print("Applying state dict...")
    final_test_model.load_state_dict(remove_module_prefix(state_dict))
    print("Moving model to device...")
    final_test_model.to(device)

    print("Loading scaler and label encoder...")
    final_scaler_test = pickle.load(open(final_scaler_path, 'rb'))
    final_le_test = pickle.load(open(final_label_encoder_path, 'rb'))

    print("Preparing test data...")
    X_test_mia = np.array(mia_test_X)
    y_test_mia = np.array(mia_test_y)

    encoded_y_test_mia = final_le_test.transform(y_test_mia)

    # Debug info
    print(f"Original test labels: {np.unique(y_test_mia)}")
    print(f"Encoded test labels: {np.unique(encoded_y_test_mia)}")
    print(f"Label range: {np.min(encoded_y_test_mia)} to {np.max(encoded_y_test_mia)}")
    print(f"N_CLASSES: {N_CLASSES}")

    scaled_X_test_mia = final_scaler_test.transform(X_test_mia)
    mia_test_data_loader = get_data_loader(scaled_X_test_mia, encoded_y_test_mia, BATCH_SIZE=BATCH_SIZE_LIST[0], shuffle=False)

    print("Running evaluation...")
    final_test_loss, final_test_acc, final_test_f1 = evaluate(final_test_model, mia_test_data_loader, criterion, device)

    print(f'Final Model Test Loss: {final_test_loss}, Test Acc: {final_test_acc}, Test F1: {final_test_f1}')

except RuntimeError as e:
    print(f"CUDA Error occurred: {e}")
    print("Attempting CPU fallback...")

    # Fallback to CPU
    device_cpu = torch.device('cpu')

    original_model = get_pretrained_harnet(class_num=N_CLASSES, model_name='harnet10')
    final_test_model = copy.deepcopy(original_model)

    state_dict = torch.load(final_model_path, weights_only=True, map_location='cpu')
    final_test_model.load_state_dict(remove_module_prefix(state_dict))
    final_test_model.to(device_cpu)  # Use CPU

    final_scaler_test = pickle.load(open(final_scaler_path, 'rb'))
    final_le_test = pickle.load(open(final_label_encoder_path, 'rb'))

    X_test_mia = np.array(mia_test_X)
    y_test_mia = np.array(mia_test_y)
    encoded_y_test_mia = final_le_test.transform(y_test_mia)
    scaled_X_test_mia = final_scaler_test.transform(X_test_mia)

    mia_test_data_loader = get_data_loader(scaled_X_test_mia, encoded_y_test_mia, BATCH_SIZE=BATCH_SIZE_LIST[0], shuffle=False)

    final_test_loss, final_test_acc, final_test_f1 = evaluate(final_test_model, mia_test_data_loader, criterion, device_cpu)

    print(f'Final Model Test Loss (CPU): {final_test_loss}, Test Acc: {final_test_acc}, Test F1: {final_test_f1}')

Downloading: "https://github.com/OxWearables/ssl-wearables/zipball/main" to /root/.cache/torch/hub/main.zip


131 Weights loaded
Loading model state dict...
Applying state dict...
Moving model to device...
Loading scaler and label encoder...
Preparing test data...
Original test labels: [ 1  2  3  4 12 13 16 17]
Encoded test labels: [0 1 2 3 4 5 6 7]
Label range: 0 to 7
N_CLASSES: 8
Running evaluation...
Evaluation set: Average loss: 1.3445, Accuracy: 523/721 (72.54%), F1-score: 0.7021

Final Model Test Loss: 1.344465445486529, Test Acc: 72.5381414701803, Test F1: 0.7021015853978003
